In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

# Lendo os dados

In [3]:
dados = pd.read_csv("dataset/sementes.csv")

dados.head()

,Área,Perímetro,Compacidade,Comprimento,Largura,Assimetria,Comprimento do sulco,Espécie
0,15.26,14.84,871.000,5.763,3.312,2.221,5.220,0
1,14.88,14.57,8.811,5.554,3.333,1.018,4.956,0
2,14.29,14.09,905.000,5.291,3.337,2.699,4.825,0
3,13.84,13.94,8.955,5.324,3.379,2.259,4.805,0
4,16.14,14.99,9.034,5.658,3.562,1.355,5.175,0


In [4]:
print(f"Linas {dados.shape[0]}")
print(f"Colunas {dados.shape[1]}")
print(f"Labels {dados['Espécie'].unique()}")

Linas 210
Colunas 8
Labels [0 1 2]


# Preparando os dados para o treinamento

In [5]:
X = dados.drop(["Espécie"], axis=1).values
y = dados["Espécie"].values

In [6]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2)

In [7]:
X_treino = torch.FloatTensor(X_treino)
X_teste = torch.FloatTensor(X_teste)
y_treino = torch.LongTensor(y_treino)
y_teste = torch.LongTensor(y_teste)

In [8]:
print(f"X: {X_treino.shape[1]}")

X: 7


# Criando a Rede

In [9]:
class Modelo(nn.Module):
    def __init__(
        self, 
        entrada=7, 
        camada_escondida1=14, 
        camada_escondida2=49,
        saida=3
    ):
        super().__init__()
        self.fc1 = nn.Linear(entrada, camada_escondida1)
        self.fc2 = nn.Linear(camada_escondida1, camada_escondida2)
        self.out = nn.Linear(camada_escondida2, saida)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [10]:
modelo_classificacao = Modelo()

In [11]:
funcao_objetivo = nn.CrossEntropyLoss()
otimizador = torch.optim.Adam(modelo_classificacao.parameters(), lr=0.01)

# Treinaamento

In [12]:
epocas = 100
custos = []

for i in range(epocas):
    y_predito = modelo_classificacao.forward(X_treino)
    custo = funcao_objetivo(y_predito, y_treino)
    custos.append(custo)

    otimizador.zero_grad()
    custo.backward()
    otimizador.step()

print(f"Loss: {custos[-1]:e}")

Loss: 2.615108e-01


In [16]:
preds = []

with torch.no_grad():
    for val in X_teste:
        y_predito = modelo_classificacao.forward(val)
        preds.append(y_predito.argmax().item())

df = pd.DataFrame({"Y": y_teste, "YHat": preds})

df["correto"] = [
    1 
    if corr == pred else 0 for corr, pred in zip(df["Y"], df["YHat"])
]

print("{:.2%}".format(sum((df["correto"] == 1)) / df.shape[0]))

85.71%


# Salvando o modelo

In [17]:
torch.save(modelo_classificacao, "modelo_treinado/modelo_classificacao.torch")